## 特征集  
创建一个包含极少特征但效果与更复杂特征集一样出色的集合。  


### 设置：  
加载相应的库和数据。  

In [54]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
1099,-117.1,32.7,37.0,2395.0,627.0,2489.0,599.0,1.6,86300.0
1794,-117.3,33.1,12.0,5080.0,814.0,1958.0,716.0,5.4,299600.0
683,-117.0,34.0,28.0,3009.0,698.0,1200.0,626.0,1.4,104600.0
11070,-121.0,37.6,36.0,826.0,167.0,432.0,150.0,2.5,103100.0
14633,-122.2,37.7,46.0,769.0,183.0,693.0,178.0,2.2,84200.0
...,...,...,...,...,...,...,...,...,...
16181,-122.5,37.7,50.0,1653.0,252.0,641.0,224.0,10.7,500001.0
7053,-118.3,34.2,39.0,1995.0,564.0,1202.0,544.0,3.6,250000.0
4195,-118.0,33.8,30.0,1859.0,462.0,1314.0,418.0,3.1,184400.0
9096,-119.0,35.4,35.0,1021.0,280.0,1258.0,239.0,1.7,48600.0


In [55]:
def preprocess_features(california_housing_dataframe):  
    #选择需要的特征 
    selected_features = california_housing_dataframe[
        ["latitude",
         "longitude",
         "housing_median_age",
         "total_rooms",
         "total_bedrooms",
         "population",
         "households",
         "median_income"]]
    processed_features = selected_features.copy()
    #创建合成特征
    processed_features["rooms_per_person"] = (
        california_housing_dataframe["total_rooms"] / california_housing_dataframe["population"])
    return processed_features

def preprocess_targets(california_housing_dataframe):
    output_targets = pd.DataFrame()
    output_targets["median_house_value"] = (
        california_housing_dataframe["median_house_value"] / 1000.0)
    return output_targets

In [56]:
#选择训练集大小，验证集大小。并输出相关信息
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

print("Training examples summary:")
display.display(training_examples.describe())
print("Valitation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Valitation targets summary:")
display.display(validation_targets.describe())

Training examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,35.6,-119.6,28.5,2652.8,540.6,1435.8,501.6,3.9,2.0
std,2.1,2.0,12.7,2248.2,430.4,1192.8,390.9,1.9,1.2
min,32.5,-124.3,1.0,2.0,1.0,3.0,1.0,0.5,0.0
25%,33.9,-121.8,18.0,1457.0,296.0,785.0,281.0,2.6,1.5
50%,34.2,-118.5,28.5,2122.5,433.0,1166.0,409.0,3.6,1.9
75%,37.7,-118.0,37.0,3148.2,647.0,1721.2,603.0,4.8,2.3
max,42.0,-114.3,52.0,37937.0,6445.0,35682.0,6082.0,15.0,55.2


Valitation examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,35.6,-119.6,28.7,2621.6,536.5,1414.5,500.3,3.9,2.0
std,2.1,2.0,12.4,2006.9,399.4,1032.0,368.9,1.9,1.0
min,32.5,-124.2,1.0,12.0,4.0,13.0,5.0,0.5,0.1
25%,33.9,-121.8,18.0,1468.8,298.0,803.0,283.0,2.6,1.5
50%,34.2,-118.5,29.0,2141.5,436.0,1168.5,409.0,3.5,1.9
75%,37.7,-118.0,37.0,3157.0,653.0,1719.0,608.0,4.8,2.3
max,41.8,-114.6,52.0,20982.0,4457.0,10329.0,4204.0,15.0,41.3


Training targets summary:


,median_house_value
count,12000.0
mean,206.8
std,115.7
min,15.0
25%,119.3
50%,179.8
75%,263.8
max,500.0


Valitation targets summary:


,median_house_value
count,5000.0
mean,208.5
std,116.7
min,17.5
25%,120.0
50%,181.3
75%,267.6
max,500.0


### 构建良好的特征集  
**相关矩阵**展现了两两比较的相关性，既包括每个特征与目标特征之间的比较，也包括每个特征与其他特征之间的比较。
在这里，相关性被定义为皮尔逊相关系数。
相关性值具有以下含义：
* -1.0：完全负相关
* 0.0：不相关
* 1.0：完全正相关

In [57]:
correlation_dataframe = training_examples.copy()
correlation_dataframe["target"] = training_targets["median_house_value"]

correlation_dataframe.corr()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person,target
latitude,1.0,-0.9,0.0,-0.0,-0.1,-0.1,-0.1,-0.1,0.1,-0.2
longitude,-0.9,1.0,-0.1,0.0,0.1,0.1,0.1,-0.0,-0.1,-0.0
housing_median_age,0.0,-0.1,1.0,-0.4,-0.3,-0.3,-0.3,-0.1,-0.1,0.1
total_rooms,-0.0,0.0,-0.4,1.0,0.9,0.9,0.9,0.2,0.1,0.1
total_bedrooms,-0.1,0.1,-0.3,0.9,1.0,0.9,1.0,-0.0,0.0,0.0
population,-0.1,0.1,-0.3,0.9,0.9,1.0,0.9,0.0,-0.1,-0.0
households,-0.1,0.1,-0.3,0.9,1.0,0.9,1.0,0.0,-0.0,0.1
median_income,-0.1,-0.0,-0.1,0.2,-0.0,0.0,0.0,1.0,0.2,0.7
rooms_per_person,0.1,-0.1,-0.1,0.1,0.0,-0.1,-0.0,0.2,1.0,0.2
target,-0.2,-0.0,0.1,0.1,0.0,-0.0,0.1,0.7,0.2,1.0


### 模型训练

In [58]:
def construct_feature_columns(input_features):
  """
  构建特征列
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [59]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """构建输入函数
    """
    
    # Convert pandas data into a dict of np arrays
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
    
    # Construct a dataset, and configure batching/repeating
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)

    # Shuffle the data, if specified
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [60]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    periods = 10
    steps_per_period = steps / periods

  # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    linear_regressor = tf.estimator.LinearRegressor(
        feature_columns=construct_feature_columns(training_examples),
        optimizer=my_optimizer
    )
    
  # Create input functions
    training_input_fn = lambda: my_input_fn(training_examples, 
                                            training_targets["median_house_value"], 
                                            batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                    training_targets["median_house_value"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["median_house_value"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    validation_rmse = []
    for period in range (0, periods):
    # Train the model, starting from the prior state.
        linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period,
    )
    # Take a break and compute predictions.
        training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
        validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, training_targets))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")

  
  # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()

    return linear_regressor

In [61]:
#使用较少的特征来训练模型
#minimal_features = [ "median_income", "latitude"
#]
#assert minimal_features, "You must select at least one feature!"

#minimal_training_examples = training_examples[minimal_features]
#minimal_validation_examples = validation_examples[minimal_features]

#train_model(
 #   learning_rate=0.02,
  #  steps=500,
   # batch_size=5,
    #training_examples=minimal_training_examples,
    #training_targets=training_targets,
    #validation_examples=minimal_validation_examples,
    #validation_targets=validation_targets)

绘制latitue和median_house_value的图形后，发现两者不存在线性关系

In [62]:
plt.scatter(training_examples["latitude"], training_targets["median_house_value"])
#plt.scatter(training_examples["median_income"], training_targets["median_house_value"])

In [69]:
LATITUDE_RANGES = zip(range(32, 44), range(33, 45))

def select_and_transform_features(source_df):
    selected_examples = pd.DataFrame()
    selected_examples["median_income"] = source_df["median_income"]
    for r in LATITUDE_RANGES:
        selected_examples["latitude_%d_to_%d" % r] = source_df["latitude"].apply(
            lambda l: 1.0 if l >= r[0] and l < r[1] else 0.0)
    return selected_examples

selected_training_examples = select_and_transform_features(training_examples)
selected_validation_examples = select_and_transform_features(validation_examples)

validation_examples


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
3856,34.0,-118.0,29.0,4943.0,674.0,1913.0,641.0,6.8,2.6
4490,34.0,-118.0,25.0,2945.0,914.0,2313.0,832.0,2.6,1.3
1243,32.9,-117.1,12.0,1474.0,364.0,1009.0,372.0,4.1,1.5
15869,37.8,-122.4,52.0,2112.0,528.0,1227.0,513.0,3.6,1.7
5686,33.8,-118.2,36.0,1697.0,550.0,1379.0,434.0,1.3,1.2
...,...,...,...,...,...,...,...,...,...
16181,37.7,-122.5,50.0,1653.0,252.0,641.0,224.0,10.7,2.6
7053,34.2,-118.3,39.0,1995.0,564.0,1202.0,544.0,3.6,1.7
4195,33.8,-118.0,30.0,1859.0,462.0,1314.0,418.0,3.1,1.4
9096,35.4,-119.0,35.0,1021.0,280.0,1258.0,239.0,1.7,0.8


In [64]:
_ = train_model(
    learning_rate=0.02,
    steps=500,
    batch_size=5,
    training_examples=selected_training_examples,
    training_targets=training_targets,
    validation_examples=selected_validation_examples,
    validation_targets=validation_targets)

Training model...
RMSE (on training data):


ValueError: Feature latitude_32_to_33 is not in features dictionary.

### 问题：为何这里的validation_example读取不出分箱数据，但是training_example却可以。